In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import gc

In [ ]:
train = pd.read_csv('../input/ashrae-energy-prediction/train.csv')
weather_train = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')
building = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv')
weather_test = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')

In [ ]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_mem_usage(train)
weather_train = reduce_mem_usage(weather_train)
building = reduce_mem_usage(building)

In [ ]:
weather_test = reduce_mem_usage(weather_test)
building = reduce_mem_usage(building) 

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train['timestamp'] = pd.to_datetime(train['timestamp'])
print(f'''{train['timestamp'].dtype}
{train['timestamp'].min()}
{train['timestamp'].max()}''')

In [ ]:
train.isnull().sum()

In [ ]:
train['meter_reading'].describe()


In [ ]:
# Calculating the zero percent consuming electricity

In [ ]:
print('Percent of zero read values:', "%.2f"%(train[train['meter_reading']== 0].shape[0] /train.shape[0]))
train[train['meter_reading'] == 0]

In [ ]:
# Calculating the percent of the zero consuming electricity buildings:

In [ ]:
print(f""""
Total number of buildings: {train['building_id'].nunique()}
Number of buildings with zero readings: {train[train['meter_reading']== 0]["building_id"].value_counts().shape[0]}
Percentage: {round(train[train['meter_reading']== 0]["building_id"].value_counts().shape[0] / train['building_id'].nunique(),2)}

Number of zero readings per building: 
{train[train['meter_reading']== 0]["building_id"].value_counts().sort_values(ascending = False)}


"""
     )


In [ ]:
(train[train['meter_reading']== 0]).groupby('building_id')['meter'].value_counts().sort_values(ascending = False)

In [ ]:
# Number of zero buliding for each buliding

In [ ]:
print('Number of zero readings for each meter:\n',train[train['meter_reading']== 0]["meter"].value_counts(),'\n')

for i in range(train["meter"].nunique()):
    percent = round(train[train['meter_reading']== 0]["meter"].value_counts()[i] /train["meter"].value_counts()[i],2)
    print(f'% of zero reads for Meter {i}: {percent}')

In [ ]:
# Defining each parameter 

In [ ]:
train['meter'] = pd.Categorical(train['meter']).rename_categories({0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'})

In [ ]:
g = sns.FacetGrid(train[train['meter_reading']== 0], col="meter",hue = 'meter',palette='coolwarm',col_wrap=2,height=3, aspect=2)
g.map(sns.histplot, 'timestamp', bins=12)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(14, 6))
sns.boxplot(y='meter', x='meter_reading', data=train, showfliers=True)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(14, 6))
sns.boxplot(y='meter', x='meter_reading', data=train[train['meter_reading'] !=  0 ], showfliers=False);

In [ ]:
sns.lineplot(data=train.groupby(['timestamp']).sum(), x="timestamp", y="meter_reading") 

In [ ]:
sns.relplot(data=(train[train['meter'].isin(['electricity','chilledwater','hotwater'])].groupby(['timestamp','meter']).sum()), x="timestamp", y="meter_reading",col="meter", hue="meter",kind="line")

In [ ]:
gc.collect()


In [ ]:
# Data Analysis in Buliding MetaData:

In [ ]:
building.head()

In [ ]:
building.info()


In [ ]:
building.describe()

In [ ]:
building.describe(include = 'O')

In [ ]:
print(f'Null value counts:\n{building.isnull().sum()}\n')
for col in list(building.columns):
    if building[col].isnull().sum() > 0:
        print(f'% of null in column {col}: {round(building[col].isnull().sum() / building.shape[0], 2 )}' )

In [ ]:
building.fillna('Missing', inplace=True)

In [ ]:
print('For Primary Usage Areas\n')
print('% of Null year_built Values Less Than 50%')
for usage in list(building['primary_use'].unique()):
    percent  = round(sum(building[building['primary_use']== usage]['year_built']== 'Missing') / building[building['primary_use']== usage].shape[0], 2)
    if percent < 0.5:
        print(f'{usage}: {percent}')
        
print('\n% of Null year_built Values Higher Than 50%')
for usage in list(building['primary_use'].unique()):
    percent  = round(sum(building[building['primary_use']== usage]['year_built']== 'Missing') / building[building['primary_use']== usage].shape[0], 2)
    if percent > 0.5:
        print(f'{usage}: {percent}')

In [ ]:
print('For Primary Usage Areas\n')
print('\n% of Null floor_count Values Less Than 50%\n')
for usage in list(building['primary_use'].unique()):
    percent  = round(sum(building[building['primary_use']== usage]['floor_count']== 'Missing') / building[building['primary_use']== usage].shape[0], 2)
    if percent < 0.5:
        print(f'{usage}: {percent}')
        
print('\n% of Null floor_count Values Higher Than 50%\n')
for usage in list(building['primary_use'].unique()):
    percent  = round(sum(building[building['primary_use']== usage]['floor_count']== 'Missing') / building[building['primary_use']== usage].shape[0], 2)
    if percent > 0.5:
        print(f'{usage}: {percent}')

In [ ]:
# Dropping floor count and year built:

In [ ]:
building.drop(['floor_count','year_built'],axis=1,inplace=True)


In [ ]:
gc.collect()

In [ ]:
# Testing weather data:

In [ ]:
weather_train.head()


In [ ]:
weather_train.info()

In [ ]:
# Calculating the timestamp:

In [ ]:
weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'])
print(f'''{weather_train['timestamp'].dtype}
{weather_train['timestamp'].min()}
{weather_train['timestamp'].max()}''')

In [ ]:
# Null value check

In [ ]:
print(f'Null value counts:\n{weather_train.isnull().sum()}\n')
for col in list(weather_train.columns):
    if weather_train[col].isnull().sum() > 0:
        print(f'% of null in column {col}: {round(weather_train[col].isnull().sum() / weather_train.shape[0], 4 )}' )

In [ ]:
weather_train.drop(['cloud_coverage','precip_depth_1_hr'], axis = 1,inplace=True)

In [ ]:
weather_train['hour'] = weather_train.timestamp.dt.hour
weather_train['month'] = weather_train.timestamp.dt.month

In [ ]:
# Using mean values to impute meteorological data:

In [ ]:
def site_mean_weather(table):
    for col in list(table.columns[table.isnull().any()]):
        imputaion = table.groupby(['site_id','hour','month'])[col].transform('mean')
        table[col].fillna(imputaion,inplace = True)
    print('Imputation with mean values is completed.')
     
site_mean_weather(weather_train)

In [ ]:
weather_train.isnull().sum()

In [ ]:
# After imputation I realize there are still missing values in sea_level_pressure column, so I droped that column to.

In [ ]:
weather_train.drop(['sea_level_pressure'], axis = 1, inplace = True)

In [ ]:
gc.collect()

In [ ]:
# Data Analysis

In [ ]:
# To evaluate all train data together train, building and weather tables are merged together and unnecessary tables are deleted.

In [ ]:
df = pd.merge(train,building, on="building_id", how="left")
df = df.merge(weather_train, on=["site_id", "timestamp"], how="left")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.nunique()

In [ ]:
df.isnull().sum()

In [ ]:
df['month'] = df.timestamp.dt.month
df['hour'] = df.timestamp.dt.hour

In [ ]:
site_mean_weather(df)

In [ ]:
for col in list(df.columns[df.isnull().any()]):    
    imputaion = df.groupby(['hour','month'])[col].transform('mean')
    df[col].fillna(imputaion,inplace = True)
print('Imputation is completed.')

In [ ]:
del train
del building
del weather_train
gc.collect()

In [ ]:
df.dtypes

In [ ]:
df[['primary_use','hour','month','site_id','building_id','wind_direction']] = df[['primary_use','hour','month','site_id','building_id','wind_direction']].astype('category')

In [ ]:
gc.collect()

In [ ]:
df['cons/sqft'] = df['meter_reading'] / df['square_feet']

In [ ]:
df.loc[(df['site_id'] == 0) & (df['meter'] == 'electricity'), 'meter_reading'] = df[(df['site_id'] == 0) & (df['meter'] == 'electricity')]['meter_reading'].apply(lambda x: x* 0.2931 )

In [ ]:
df['day'] = df.timestamp.dt.year

In [ ]:
df = reduce_mem_usage(df)
gc.collect()

In [ ]:
# Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4),constrained_layout=True)
fig.suptitle('Building Counts')
sns.barplot(ax=axes[0],y="site_id", x='building_id', data=df.groupby(['site_id'])['building_id'].nunique().reset_index())
axes[0].set(xlabel = 'Building Count', ylabel = 'Site id')  
sns.barplot(ax=axes[1],y="primary_use", x='building_id', data=df.groupby(['primary_use'])['building_id'].nunique().reset_index())
axes[1].set(xlabel = 'Building Count', ylabel = 'Primary Use')

In [ ]:
gc.collect()


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 16),constrained_layout=True)
fig.suptitle('Energy Consumption for Primary Use')
sns.boxplot(ax=axes[0, 0], y='primary_use', x='meter_reading', data=df, showfliers=True)
sns.boxplot(ax=axes[0, 1], y='primary_use', x='meter_reading', data=df, showfliers=False)
sns.boxplot(ax=axes[1, 0], y='primary_use', x='cons/sqft', data=df, showfliers=True)
sns.boxplot(ax=axes[1, 1], y='primary_use', x='cons/sqft', data=df, showfliers=False);

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 16),constrained_layout=True)
fig.suptitle('Energy Consumption for Sites')
sns.boxplot(ax=axes[0, 0], y='site_id', x='meter_reading', data=df, showfliers=True)
sns.boxplot(ax=axes[0, 1], y='site_id', x='meter_reading', data=df, showfliers=False)
sns.boxplot(ax=axes[1, 0], y='site_id', x='cons/sqft', data=df, showfliers=True)
sns.boxplot(ax=axes[1, 1], y='site_id', x='cons/sqft', data=df, showfliers=False);

In [ ]:
gc.collect()

In [ ]:
df['day'] = df.timestamp.dt.day

In [ ]:
df[df['meter']=='electricity'].groupby(['site_id','month','day','building_id'])['meter_reading'].sum().sort_values(ascending = False).reset_index().head(100)

In [ ]:
df[df['meter']=='electricity'].groupby(['site_id','month','day','building_id'])['cons/sqft'].sum().sort_values(ascending = False).reset_index().head(100)

In [ ]:
df.groupby(['site_id'])['meter_reading'].sum().sort_values(ascending = False).reset_index()
gc.collect()

In [ ]:
gc.collect()

In [ ]:
# Removing Outliers

In [ ]:
total = 0
print('Outlier distribution in meter types')
for col in list(df['meter'].unique()):
    r = np.percentile(df[df['meter'] == col]['cons/sqft'],75) + 1.5 * (np.percentile(df[df['meter'] == col]['cons/sqft'],75) - np.percentile(df[df['meter'] == col]['cons/sqft'],25))
    print(f'''Percentage of outliers in {col} to all readings: { round(df[(df['meter'] == col) & (df['cons/sqft'] > r)].shape[0]/ df.shape[0],5)}''')
    total +=  df[(df['meter'] == col) & (df['cons/sqft'] > r)].shape[0]
print(f'Total fraction of outliers {round(total / df.shape[0],5)}')
gc.collect()

In [ ]:
total = 0
print('Outlier distribution in building types')
for col in list(df['primary_use'].unique()):
    r = np.percentile(df[df['primary_use'] == col]['cons/sqft'],75) + 1.5 * (np.percentile(df[df['primary_use'] == col]['cons/sqft'],75) - np.percentile(df[df['primary_use'] == col]['cons/sqft'],25))
    print(f'''Percentage of outliers in {col} to all readings: { round(df[(df['primary_use'] == col) & (df['cons/sqft'] > r)].shape[0]/ df.shape[0],5)}''')
    total +=  df[(df['primary_use'] == col) & (df['cons/sqft'] > r)].shape[0]
print(f'Total fraction of outliers {round(total / df.shape[0],5)}')
gc.collect()

In [ ]:
total = 0
print('Outlier distribution in building types')
for col in list(df['site_id'].unique()):
    r = np.percentile(df[df['site_id'] == col]['cons/sqft'],75) + 1.5 * (np.percentile(df[df['site_id'] == col]['cons/sqft'],75) - np.percentile(df[df['site_id'] == col]['cons/sqft'],25))
    print(f'''Percentage of outliers in {col} to all readings: { round(df[(df['site_id'] == col) & (df['cons/sqft'] > r)].shape[0]/ df.shape[0],5)}''')
    total +=  df[(df['site_id'] == col) & (df['cons/sqft'] > r)].shape[0]
print(f'Total fraction of outliers {round(total / df.shape[0],5)}')
gc.collect()

In [ ]:
df.groupby(['month','day','building_id','meter'])['meter_reading'].sum().sort_values(ascending = False).reset_index().head()

In [ ]:
df.groupby(['month','day','building_id','meter'])['meter_reading'].sum().sort_values(ascending = False).reset_index().head(100)['building_id'].value_counts()

In [ ]:
gc.collect()

In [ ]:
df.groupby(['building_id','site_id','meter'])['meter_reading'].sum().sort_values(ascending = False).reset_index().head(10)

In [ ]:
df.groupby(['building_id','site_id','meter'])['cons/sqft'].sum().sort_values(ascending = False).reset_index().head(10)